# do analysis on same trials

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["OMP_NUM_THREADS"] = "1"
import seaborn as sns 
import ast
from scipy.stats import pearsonr

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.stats import pearsonr
from statsmodels.stats.multitest import fdrcorrection
import warnings
warnings.filterwarnings("ignore")
from functools import reduce

In [2]:
output_dir = r"15_RL_agent_TDlearn_output_behavior_same_trials"
os.makedirs(output_dir, exist_ok=True)


folder_path_participants = 'data_risk_added'
folder_path_colors = '11_RL_agent_TDlearn_output/model_behavior'
folder_path_numbers = '12_RL_agent_TDlearn_output/model_behavior'
folder_path_colors_numbers = '13_RL_agent_TDlearn_output/model_behavior'


df_participants = []
df_colors = []
df_numbers = []
df_colors_numbers = []
clean_names= []

def find_matching_csv(folder_path, df_list):
            for csv_file in os.listdir(folder_path):
                if clean_name in csv_file and csv_file.endswith('.csv'):
                    csv_path = os.path.join(folder_path, csv_file)
                    df_csv = pd.read_csv(csv_path)
                    df_list.append(df_csv)





for file_name in os.listdir(folder_path_participants):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path_participants, file_name)
        df = pd.read_excel(file_path)
        df = df[df['outcome'].str.lower() != 'na'].reset_index(drop=True)  
        df_participants.append(df)

        clean_name = file_name.removeprefix("task_data_").removesuffix(".xlsx")
        clean_names.append(clean_name)


        find_matching_csv(folder_path_colors, df_colors)
        find_matching_csv(folder_path_numbers, df_numbers)
        find_matching_csv(folder_path_colors_numbers, df_colors_numbers)


In [3]:
df_participants[2].head(5)

,arrowRT,distribution,interTrialInterval,outcome,myCard,yourCard,spaceRT,totalReward,trialIndex,trialType,choice,block,timeoutRepeat,risk
0,1067,uniform,894,win,3,9,4575,10.5,0,response,arrowdown,1,0,0.250
1,237,uniform,925,lose,2,1,1903,10.0,1,response,arrowdown,1,0,0.125
2,231,uniform,973,win,8,2,2130,10.5,2,response,arrowup,1,0,0.125
3,602,uniform,768,win,7,2,1537,11.0,3,response,arrowup,1,0,0.250
4,273,uniform,937,win,9,8,1809,11.5,4,response,arrowup,1,0,0.000


In [4]:
df_colors[2].head(5)

,model_choices,participant_choices,model_total_reward,participant_total_reward,q_val
0,1,0,9.5,10.5,"[[0.225697305012495, 0.017228503202055424], [0..."
1,0,0,9.0,10.0,"[[-0.08776403092966037, 0.017228503202055424],..."
2,0,1,8.5,10.5,"[[-0.08776403092966037, 0.22575924638632577], ..."
3,0,1,8.0,11.0,"[[-0.08776403092966037, 0.3442161738184122], [..."
4,1,1,8.5,11.5,"[[-0.08776403092966037, 0.4115062215225583], [..."


In [5]:
df_numbers[2].head()

,model_choices,participant_choices,model_total_reward,participant_total_reward,q_val
0,0,0,10.5,10.5,"[[-0.005781484717304979, -0.000360978904806554..."
1,1,0,11.0,10.0,"[[-0.005781484717304979, -0.000360978904806554..."
2,1,1,11.5,10.5,"[[-0.005781484717304979, -0.000360978904806554..."
3,1,1,12.0,11.0,"[[-0.005781484717304979, -0.000360978904806554..."
4,0,1,11.5,11.5,"[[-0.005781484717304979, -0.000360978904806554..."


In [6]:
df_colors_numbers[2].head(5)

,model_choices,participant_choices,model_total_reward,participant_total_reward,q_val
0,1,0,9.5,10.5,"[[[-0.020376703070331587, -0.00073429560613807..."
1,0,0,9.0,10.0,"[[[-0.020376703070331587, -0.00073429560613807..."
2,0,1,8.5,10.5,"[[[-0.020376703070331587, -0.00073429560613807..."
3,1,1,9.0,11.0,"[[[-0.020376703070331587, -0.00073429560613807..."
4,1,1,9.5,11.5,"[[[-0.020376703070331587, -0.00073429560613807..."


In [ ]:
from functools import reduce

# Step 1: Count (myCard, distribution) occurrences with named columns
df_counts = []
for df, name in zip(df_participants, clean_names):
    counts = df.groupby(['myCard', 'distribution']).size().reset_index(name=f'count_{name}')
    df_counts.append(counts)

# Step 2: Merge all on ['myCard', 'distribution']
common_trials = reduce(lambda left, right: pd.merge(left, right, on=['myCard', 'distribution']), df_counts)

# Step 3: Compute the minimum count for each common trial
count_cols = [col for col in common_trials.columns if col.startswith('count_')]
common_trials['min_count'] = common_trials[count_cols].min(axis=1)

# Step 4: Collect matching indices per df
matching_indices_list = []
for df, name in zip(df_participants, clean_names):
    indices = []
    for _, row in common_trials.iterrows():
        mask = (df['myCard'] == row['myCard']) & (df['distribution'] == row['distribution'])
        matches = df[mask]
        indices.extend(matches.index[:int(row[f'count_{name}'])])
    matching_indices_list.append(sorted(indices))



total_common_trials = common_trials['min_count'].sum()
print("Total number of common trials:", total_common_trials)


Total number of common trials: 181


In [8]:
common_trials

,myCard,distribution,count_07_11_2024_17_23_43,count_08_11_2024_13_03_29,count_11_11_2024_16_46_44,count_12_11_2024_00_15_17,count_13_11_2024_10_46_21,count_13_11_2024_14_45_52,count_14_11_2024_21_46_47,count_15_11_2024_11_43_48,...,count_25_11_2024_07_37_11,count_25_11_2024_12_11_10,count_25_11_2024_18_41_38,count_25_11_2024_20_12_41,count_26_03_2025_16_21_25,count_26_11_2024_10_53_23,count_26_11_2024_14_31_40,count_28_11_2024_12_21_16,count_28_11_2024_22_38_25,min_count
0,1,high,3,3,2,1,3,1,3,2,...,3,1,2,2,2,2,2,1,2,1
1,1,low,15,15,17,17,20,17,16,15,...,16,16,18,20,17,15,23,16,16,13
2,1,uniform,11,7,11,8,9,9,9,8,...,10,9,11,8,7,12,9,10,11,7
3,2,high,4,2,5,4,2,4,4,4,...,5,3,4,3,4,4,5,5,3,2
4,2,low,16,14,19,15,15,16,15,18,...,17,18,16,16,17,16,15,15,17,11
5,2,uniform,9,8,9,9,10,9,8,8,...,9,9,9,8,9,11,9,9,12,7
6,3,high,5,11,5,6,8,6,6,6,...,5,7,6,8,3,5,8,3,8,3
7,3,low,11,16,11,18,14,14,13,16,...,13,14,12,15,13,15,14,17,16,11
8,3,uniform,9,8,8,10,12,6,9,11,...,9,12,9,10,7,7,12,11,8,6
9,4,high,7,6,9,7,9,7,9,6,...,9,11,9,7,8,12,9,10,8,5
